# MODELO VOOTING (VERSION: 1)

In [37]:
# Importamos las librerias a usar
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

import sklearn.metrics as metrics
from sklearn.metrics import recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [16]:
#Prueba 2
# 1 Cost_of_the_Product, Discount_offered)
# 2 Discount_offered, Weigth_gms)
# Weigth <---> Discount <---> Cost_Product
# Weigth <---> Customer_care_Calls <---> Cost_Productos

In [17]:
data_train = pd.read_csv('E-Commerce_train.csv',delimiter = ';')
columnas_descartables = ['ID','Product_importance','Warehouse_block','Gender','Prior_purchases','Mode_of_Shipment','Customer_rating']
data_train = data_train.drop(columns=columnas_descartables)
data_train.head(3)


,Customer_care_calls,Cost_of_the_Product,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,4,177,44,1233,1
1,4,216,59,3088,1
2,2,183,48,3374,1


In [18]:
# Definimos atributos y variable a predecir

X = data_train[['Customer_care_calls', 'Cost_of_the_Product', 'Discount_offered', 'Weight_in_gms']]

Y = data_train['Reached.on.Time_Y.N']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [20]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

In [38]:
# Instanciamos 3 modelos



desc_Class = DecisionTreeClassifier()

k_neigh = KNeighborsClassifier(n_neighbors=4,weights = 'distance',leaf_size=20)

svc_clf = SVC(C=1, kernel='sigmoid', gamma='auto')

naive_clf = GaussianNB()

log_clf = LogisticRegression(C=1, solver='liblinear')

In [39]:
# Los entrenamos, predecimos y evaluamos
import time

comienzo = time.time()
for clf in (log_clf, svc_clf, naive_clf, desc_Class, k_neigh):
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    
fin = time.time() - comienzo

print(f'Tardó {round(fin)} segundos')

RandomForestClassifier 0.6905555555555556
SVC 0.6061111111111112
GaussianNB 0.6722222222222223
DecisionTreeClassifier 0.7027777777777777
KNeighborsClassifier 0.7011111111111111
Tardó 3 segundos


In [41]:
voting_clf_hard = VotingClassifier(estimators=[('lr', clf_forest), 
                                               ('svc', svc_clf),
                                               ('knei', k_neigh),
                                               ('tree',desc_Class), 
                                               ('naive', naive_clf)],
                                   voting='hard')

voting_clf_hard.fit(x_train, y_train)
y_pred = voting_clf_hard.predict(x_test)

In [42]:
print('Accuracy en test:', accuracy_score(y_test, y_pred))

Accuracy en test: 0.7033333333333334


In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.73      0.63       634
           1       0.82      0.69      0.75      1166

    accuracy                           0.70      1800
   macro avg       0.69      0.71      0.69      1800
weighted avg       0.73      0.70      0.71      1800



In [44]:
# Datos de testeo para comprobacion de accuracy y reacall de dashboard
data_test = pd.read_csv('E-Commerce_test.csv',delimiter = ';')
columnas_descartables = ['ID','Gender','Prior_purchases','Mode_of_Shipment','Customer_rating']
data_test = data_test.drop(columns=columnas_descartables)
data_test.Warehouse_block = data_test['Warehouse_block'].map({'A':1,'B':2,'C':3,'D':4,'F':5})
data_test.Product_importance = data_test['Product_importance'].map({'low':1,'medium':2,'high':3})

In [45]:
X_test_pred = data_test[['Customer_care_calls', 'Cost_of_the_Product', 'Discount_offered', 'Weight_in_gms']]
sc_X_test = StandardScaler()
X_test_pred = sc_X_test.fit_transform(X_test_pred)


In [46]:
Y_test = voting_clf_hard.predict(X_test_pred)

In [47]:
Y_pred = pd.DataFrame(Y_test)
Y_pred = Y_pred.rename(columns={0: 'pred'})

In [48]:
Y_pred.to_csv('AlexCh0693.csv',index =  False)